In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# Import Dataset, Processing

In [2]:
DF = pd.read_csv('gurobiTime.csv',index_col=0)
DF = DF.drop(['player.1'],axis=1)


positionDF = []
num_positions = len(DF['position'].unique())


for i in DF['position'].unique():
    positionDF.append( (DF[DF['position'] == i]) )

    

Make Gurobi datastructures to help represent the problem

We create dictionaries for every single variable for each player, to allow for efficient access and ease of use.  The keys are Players, the tupleList objects are returnPlayer, playerVariance, playerCost , and position


QB, RB, RB, WR, WR, WR, TE, D, K 



In [3]:
bigDict = {}
for df_iter in range(len(positionDF)):
    
    currPosition = positionDF[df_iter]  
    for player in currPosition.index:
        returnPlayer = currPosition.loc[player]['mean']
        playerVariance = currPosition.loc[player]['variance']
        playerCost = currPosition.loc[player]['cost']
        position = currPosition.loc[player]['position']
        bigDict[player] = [returnPlayer, playerVariance, playerCost , position]
         
player, returnPlayer, playerVariance, playerCost , position = gp.multidict(bigDict)


positionPlayerConstraintVars = gp.tuplelist( [(position.select(playr)[0],playr) for playr in player])

playerDecisionVars = gp.tuplelist( [ (choice, returnPlayer.select(choice)[0], playerVariance.select(choice)[0])  for choice in player ] )

positionConstraints = gp.tupledict( { 'QB':1, "RB":2,"WR":3, 'TE':1,"DEF":1, 'FLEX':1})#'K':0} )




In [5]:
positionPlayerConstraintVars


# Constraints: assign exactly shiftRequirements[s] workers to each shift s

# reqCts = {}
# for s in shifts:
#   reqCts[s] = m.addConstr(
#        gp.quicksum(x[w,s] for w,s in availability.select('*', s)) ==
#        shiftRequirements[s], s)
positionConstraints

{'QB': 1, 'RB': 2, 'WR': 3, 'TE': 1, 'DEF': 1, 'FLEX': 1}

In [9]:
try:
    m = gp.Model('QMIP')
    #add binary decision variables on whether 
    
    
    #add decision variables
    players = m.addVars( playerDecisionVars, vtype=GRB.BINARY, name='players') 
    
    
    
    #add position constraint. We set upper bound to be one and keep the constraint a continuous variable since the 
    #model will keep integer solutions
    
    #positionConstratint = m.addVars(positionConstraints, ub =1 , name='positionConstr')
    
    
    # set objective funcition
    m.setObjective(  - gp.quicksum( returnPlayer[choice] *players[choice,retrn,var] for choice, retrn, var in players ) 
                   + gp.quicksum( playerVariance[choice] *players[choice,retrn,var] for choice, retrn, var in players ) 
                   ,GRB.MINIMIZE)
    
    #constraint: sum of each position must be a constraint:
    #for pos in DF['position'].unique():
        
        
    posConstrs = {} 
    for pos in DF['position'].unique():
        if 'pos' == 'FLEX':
            continue
        players_in_pos = [playr for playr in player if position[playr] == pos]
        
        posConstrs[pos] = m.addConstr( gp.quicksum( [ players.select(playr)[0] for playr in player if position[playr] == pos ] ) == positionConstraints[pos] )
        
  #  modelPositionConstr =  m.addConstrs(  players.sum(position[choice] ) ==  positionConstraints [ position[choice] ] for choice ,_,_ in players  ) 
    
    #salary constraint
    costConstr = m.addConstr(gp.quicksum( [ playerCost[playr] *players.select(playr)[0] for playr in player ]) <= 50000 )
    

    #flex constraints
  #  flexConstr = m.addConstr(gp.quicksum([ players.prod(players.select(playr),'*') for playr in player if playr == 'WR' ] ) == 1 )
    
    m.update()
    
    m.optimize()
    
    foundPlayers = m.X
    
    
    teamRoster = [ player[i] for i in range(len(player)) if foundPlayers[i]==1 ]
    print('\noptimal team roster: ',teamRoster)
    
    sum = 0
    for i in teamRoster:
        sum += playerCost[i] 
    sum 


    constr = { 'QB':1, "RB":3,"WR":3, 'TE':1,"DEF":1} 

    for pos in DF['position'].unique():
        for i in teamRoster:
            if position[i] == pos:
                constr[pos] -= 1
    for key,value in constr.items():
        if value != 0:
            print('failure to meet pos constraint:',key)
    print('valid roster, cost =',sum)    
    
    print('\n expected return - risk :',m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 6 rows, 478 columns and 956 nonzeros
Model fingerprint: 0xb96f297f
Variable types: 0 continuous, 478 integer (478 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [4e-14, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Found heuristic solution: objective 340.4580800
Presolve removed 1 rows and 292 columns
Presolve time: 0.00s
Presolved: 5 rows, 186 columns, 364 nonzeros
Found heuristic solution: objective -13.6987400
Variable types: 0 continuous, 186 integer (186 binary)
Found heuristic solution: objective -13.8587400

Root relaxation: cutoff, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       -13.85874  -13.85874  0.00%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.04 secon

In [7]:
positionConstraints['FLEX']

1

In [8]:
gp.quicksum([ players.prod(players.select(playr),'*')[ for playr in player if playr == 'WR' ] ) == 1 

SyntaxError: invalid syntax (<ipython-input-8-3c4b6a8cf8f2>, line 1)

In [ ]:
gp.quicksum( [   playerCost[playr] *players.select(playr)[0] for playr in player ]) <= 50000

In [ ]:
teamRoster

sum = 0
for i in teamRoster:
    sum += playerCost[i] 
sum 


constr = { 'QB':1, "RB":3,"WR":3, 'TE':1,"DEF":1} 

for pos in DF['position'].unique():
    for i in teamRoster:
        if position[i] == pos:
            constr[pos] -= 1
for key,value in constr.items():
    if value != 0:
        print('failure to meet pos constraint:',key)
print('valid roster, cost =',sum)
